In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
id = '1J8brwnHgopc8k_il49FEAB_3BdQvdCAU'  # 共有リンクで取得した id= より後の部分
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('lib.zip')

In [0]:
ls

In [0]:
import os
# フォルダ「data」が存在しない場合は作成する
data_dir = "./lib/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [0]:
    import zipfile

    save_path = "lib.zip"
    
    # ZIPファイルを読み込み
    zip = zipfile.ZipFile(save_path)
    zip.extractall("lib")  # ZIPを解凍
    zip.close()  # ZIPファイルをクローズ

    # ZIPファイルを消去
    os.remove(save_path)

In [0]:
id = '1eDylstW-XZ7tfPmUrG__hEsWFxTWEMZE'  # 共有リンクで取得した id= より後の部分
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('stanford-dogs-dataset.zip')

In [0]:
import os
# フォルダ「data」が存在しない場合は作成する
data_dir = "./data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [0]:
    import zipfile

    save_path = "stanford-dogs-dataset.zip"
    
    # ZIPファイルを読み込み
    zip = zipfile.ZipFile(save_path)
    zip.extractall("data")  # ZIPを解凍
    zip.close()  # ZIPファイルをクローズ

    # ZIPファイルを消去
    os.remove(save_path)

In [9]:
%cd data/images/Images

/content/data/images/Images


In [0]:
os.mkdir("./train/")
os.mkdir("./val/")

In [0]:
import glob
import shutil

foruda = os.listdir(path='.')
path = os.getcwd()  

for a in foruda:
 if a.find("-") != -1:
  files = os.listdir(path + "/" + a)  
  count = len(files)  
  files = glob.glob("./"+a +"/*")
  i = 0
  for file in files:
    i = 1 + i
    if i < count*1/4:
     data_dir = "./val/" + a
     if not os.path.exists(data_dir):
      os.mkdir(data_dir)
     shutil.move(file, './val/'+ a + "/")
    else:
     data_dir = "./train/" + a
     if not os.path.exists(data_dir):
      os.mkdir(data_dir)
     shutil.move(file, './train/'+ a + "/")
  shutil.rmtree("./"+a)

In [12]:
cd train

/content/data/images/Images/train


In [13]:
cd ../val

/content/data/images/Images/val


In [15]:
ls

adc.json  data/  lib/  sample_data/


モデル適用

In [0]:
import glob
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms

torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [17]:
use_pretrained = True
net = models.vgg16(pretrained=use_pretrained)

net.classifier[6] = nn.Linear(in_features=4096, out_features=120)

net.train()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:22<00:00, 24.1MB/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [0]:
criterion = nn.CrossEntropyLoss()

In [19]:
params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight","classifier.0.bias","classifier.3.weight","classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight","classifier.6.bias"]

for name, param in net.named_parameters():
  if update_param_names_1[0] in name:
    param.requires_grad = True
    params_to_update_1.append(param)
    print("params_to_update_1に格納:", name)

  elif name in update_param_names_2:
    param.requires_grad = True
    params_to_update_2.append(param)
    print("params_to_update_2に格納：", name)

  elif name in update_param_names_3:
    param.requires_grad = True
    params_to_update_3.append(param)
    print("params_to_update_3に格納：", name)
  
  else:
    param.requires_grad = False
    print("勾配計算なし。学習しない:", name)

params_to_update_1に格納: features.0.weight
params_to_update_1に格納: features.0.bias
params_to_update_1に格納: features.2.weight
params_to_update_1に格納: features.2.bias
params_to_update_1に格納: features.5.weight
params_to_update_1に格納: features.5.bias
params_to_update_1に格納: features.7.weight
params_to_update_1に格納: features.7.bias
params_to_update_1に格納: features.10.weight
params_to_update_1に格納: features.10.bias
params_to_update_1に格納: features.12.weight
params_to_update_1に格納: features.12.bias
params_to_update_1に格納: features.14.weight
params_to_update_1に格納: features.14.bias
params_to_update_1に格納: features.17.weight
params_to_update_1に格納: features.17.bias
params_to_update_1に格納: features.19.weight
params_to_update_1に格納: features.19.bias
params_to_update_1に格納: features.21.weight
params_to_update_1に格納: features.21.bias
params_to_update_1に格納: features.24.weight
params_to_update_1に格納: features.24.bias
params_to_update_1に格納: features.26.weight
params_to_update_1に格納: features.26.bias
params_to_update_1に格納: f

In [0]:
optim = optim.SGD([
  {'params': params_to_update_1, 'lr': 1e-4},
  {'params': params_to_update_2, 'lr': 5e-4},
  {'params': params_to_update_3, 'lr': 1e-3}
], momentum=0.9)

In [21]:
%%time
! npm install -g localtunnel
!python3 -m pip install visdom
!python3 -m visdom.server -port 8076 >> visdomlog.txt 2>&1 &
!lt --port 8076 >> url.txt 2>&1 &

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.0
added 35 packages from 21 contributors in 1.62s
     |████████████████████████████████| 686kB 33.8MB/s 
     |████████████████████████████████| 204kB 53.0MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-cp36-none-any.whl size=655252 sha256=fab6a38a26857ad10b1e2a29a8e74a6384be10ee108738dad770e50fd0faa354
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5711 sha256=5526ee57ec39550a2c98499a4c50bbb0fc7b68e2376e9c131989d583725ebdb9
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built visdom torchfile
CPU times: user 112 ms, sys: 94.1 ms, total: 206 ms
Wall time: 17.7 s


In [22]:
%%time
import time
time.sleep(5)
! cat url.txt

CPU times: user 17.1 ms, sys: 19 ms, total: 36.1 ms
Wall time: 6.99 s


In [0]:
from lib.lib import network_finetuning as network
from lib.lib import architecture
from lib.lib import dataloader as dl
from lib.lib import optimizer
from lib.lib import trainer

import torchvision

In [24]:
%%time
batch_size = 128
train_dataset = torchvision.datasets.ImageFolder(root='./data/images/Images/train', transform=dl.simple_transform(resize=224))
test_dataset = torchvision.datasets.ImageFolder(root='./data/images/Images/val', transform=dl.simple_transform(resize=224))
data_loader = dl.DataLoader(train_dataset, test_dataset, batch_size=batch_size)

CPU times: user 56.5 ms, sys: 11.6 ms, total: 68 ms
Wall time: 73.5 ms


In [25]:
%%time
model = architecture.CNN_Architecture(net)

CPU times: user 342 µs, sys: 42.1 ms, total: 42.4 ms
Wall time: 73.8 ms


In [29]:
%%time
model.train(data_loader, criterion=criterion, optimizer=optim, epoch_count=50, is_inception=False)

Setting up a new session...


cuda
----------
Epoch 1/50 2020-01-22 07:27:44.181962
val Loss: 4.8180 Acc: 0.0063
----------
Epoch 2/50 2020-01-22 07:28:18.655047
train Loss: 5.0908 Acc: 0.0005
val Loss: 4.6061 Acc: 0.1323
----------
Epoch 3/50 2020-01-22 07:33:18.954830
train Loss: 4.5235 Acc: 0.0980
val Loss: 3.6709 Acc: 0.4676
----------
Epoch 4/50 2020-01-22 07:38:18.808833
train Loss: 3.3996 Acc: 0.1815
val Loss: 2.5614 Acc: 0.3701
----------
Epoch 5/50 2020-01-22 07:43:19.579534
train Loss: 2.4493 Acc: 0.3378
val Loss: 2.4764 Acc: 0.3689
----------
Epoch 6/50 2020-01-22 07:48:21.195079
train Loss: 1.8368 Acc: 0.4878
val Loss: 2.6857 Acc: 0.3597
----------
Epoch 7/50 2020-01-22 07:53:22.845653
train Loss: 1.6470 Acc: 0.5334
val Loss: 3.0976 Acc: 0.3338
----------
Epoch 8/50 2020-01-22 07:58:25.034681
train Loss: 1.6121 Acc: 0.5499
val Loss: 3.5048 Acc: 0.3111
----------
Epoch 9/50 2020-01-22 08:03:26.461613
train Loss: 1.4871 Acc: 0.5689
val Loss: 3.0663 Acc: 0.3604
----------
Epoch 10/50 2020-01-22 08:08:28.72

KeyboardInterrupt: ignored